In [ ]:
from autumn.tools.project import get_project
from matplotlib import pyplot
from autumn.tools.plots.utils import REF_DATE
# from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print

from autumn.tools.plots.uncertainty.plots import _get_target_values, _plot_targets_to_axis

In [ ]:
project = get_project("sm_sir", "hanoi")

In [ ]:
update_params = {
#     "contact_rate": 0.5,
#     "infectious_seed": 2000.,
#     "detect_prop": 0.19060505331156416,
#     "testing_to_detection.assumed_tests_parameter": 0.001138495394924509,
#     "testing_to_detection.assumed_cdr_parameter": 0.0363758883951254,
#     "immunity_stratification.prop_immune": 0.5,
#     "immunity_stratification.prop_high_among_immune": 0.5,
#     "prop_icu_among_hospitalised": 0.02,
#     "age_stratification.prop_hospital.multiplier": 0.075,
#     "age_stratification.cfr.multiplier": 0.5,
#     "voc_emergence.omicron.new_voc_seed.start_time": 672,
#     "voc_emergence.omicron.relative_latency": 0.5, # 0.6667
#     "voc_emergence.omicron.hosp_protection": 0.65,
#     "voc_emergence.omicron.death_protection": 0.8,
#     "voc_emergence.omicron.contact_rate_multiplier": 2.4,
#     "voc_emergence.omicron.relative_active_period": 0.3,
}
params = project.param_set.baseline.update(update_params, calibration_format=True)


In [ ]:
pretty_print(params)

In [ ]:
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [ ]:
outputs = ["prop_ever_infected", 
           "hospital_occupancy", 
           "infection_deaths",
           "notifications"] #, "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"]
for output in outputs:
    fig = pyplot.figure(figsize=(12, 8))
    pyplot.style.use("ggplot")
    axis = fig.add_subplot()

    axis = derived_df[output].plot()
    axis.set_title(output)
    
    if output in project.plots:
        values, times = _get_target_values(project.plots, output)        
        date_times = pd.to_datetime(times, origin="01Jan2020",unit="D") 
        _plot_targets_to_axis(axis, values, date_times, on_uncertainty_plot=True)

